In [2]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew,kurtosis
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [3]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\train.csv',encoding='big5')
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\test.csv',encoding='big5',header=None)

In [4]:
train_data1=train_data.replace('NR',0)
train_data1=train_data1.iloc[:,3:]

In [5]:
year_data={}
for m in range(12):
    month_data=np.empty((18,20*24))
    for d in range(20):
        month_data[:,d*24:(d+1)*24]=train_data1.iloc[m*20*18+d*18:m*20*18+(d+1)*18,:]
    year_data[m]=month_data

In [6]:
year_data1=year_data.copy()

In [7]:
month_list=[]
for i in range(12):
    month_list.append(pd.DataFrame(year_data1[i]))
all_train_data1=pd.concat(month_list,axis=1).T

In [8]:
all_train_data1.shape

(5760, 18)

In [9]:
all_train_data2=pd.DataFrame()
for i in range(all_train_data1.shape[1]):
    all_train_data2[i]=(all_train_data1[i]-all_train_data1[i].mean()) / all_train_data1[i].std()

In [10]:
all_train_data1.T.shape

(18, 5760)

In [11]:
all_train_data1.T.shape

(18, 5760)

In [12]:
all_train_data1.T.iloc[:10,:]

,0,1,2,3,4,5,6,7,8,9,...,470,471,472,473,474,475,476,477,478,479
0,14.00,14.00,14.00,13.00,12.00,12.00,12.00,12.00,15.00,17.00,...,19.00,18.00,17.00,16.0,15.00,14.00,13.00,13.00,13.00,13.00
1,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,...,1.70,1.70,1.70,1.8,1.80,1.80,1.80,1.80,1.80,1.80
2,0.51,0.41,0.39,0.37,0.35,0.30,0.37,0.47,0.78,0.74,...,0.24,0.27,0.31,0.5,0.55,0.48,0.47,0.51,0.57,0.56
3,0.20,0.15,0.13,0.12,0.11,0.06,0.10,0.13,0.26,0.23,...,0.08,0.11,0.23,0.3,0.20,0.15,0.16,0.16,0.19,0.19
4,0.90,0.60,0.50,1.70,1.80,1.50,1.90,2.20,6.60,7.90,...,1.30,1.30,0.90,1.1,1.30,1.00,1.50,1.50,1.10,1.30
5,16.00,9.20,8.20,6.90,6.80,3.80,6.90,7.80,15.00,21.00,...,6.10,8.00,11.00,18.0,19.00,13.00,13.00,13.00,13.00,14.00
6,17.00,9.80,8.70,8.60,8.50,5.30,8.80,9.90,22.00,29.00,...,7.40,9.30,11.00,19.0,21.00,14.00,15.00,15.00,14.00,15.00
7,16.00,30.00,27.00,23.00,24.00,28.00,24.00,22.00,21.00,29.00,...,37.00,39.00,37.00,26.0,18.00,18.00,14.00,13.00,13.00,13.00
8,56.00,50.00,48.00,35.00,25.00,12.00,4.00,2.00,11.00,38.00,...,8.00,13.00,28.00,39.0,45.00,54.00,66.00,50.00,32.00,22.00
9,26.00,39.00,36.00,35.00,31.00,28.00,25.00,20.00,19.00,30.00,...,1.00,7.00,24.00,37.0,49.00,37.00,28.00,17.00,24.00,29.00


In [13]:
all_train_data1.T.iloc[:10,475:]

,475,476,477,478,479,0,1,2,3,4,...,470,471,472,473,474,475,476,477,478,479
0,14.00,14.00,14.00,13.00,13.00,15.00,14.00,14.00,14.00,14.00,...,19.00,18.00,17.00,16.0,15.00,14.00,13.00,13.00,13.00,13.00
1,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.70,1.80,1.70,...,1.70,1.70,1.70,1.8,1.80,1.80,1.80,1.80,1.80,1.80
2,0.35,0.33,0.34,0.41,0.43,0.27,0.26,0.25,0.26,0.24,...,0.24,0.27,0.31,0.5,0.55,0.48,0.47,0.51,0.57,0.56
3,0.17,0.15,0.11,0.12,0.13,0.05,0.05,0.04,0.05,0.05,...,0.08,0.11,0.23,0.3,0.20,0.15,0.16,0.16,0.19,0.19
4,0.70,1.00,0.70,0.60,0.40,0.90,0.90,1.00,1.60,1.30,...,1.30,1.30,0.90,1.1,1.30,1.00,1.50,1.50,1.10,1.30
5,9.70,9.40,8.70,8.70,6.70,4.80,4.40,3.90,2.40,2.80,...,6.10,8.00,11.00,18.0,19.00,13.00,13.00,13.00,13.00,14.00
6,10.00,10.00,9.40,9.30,7.20,5.70,5.30,4.90,4.00,4.10,...,7.40,9.30,11.00,19.0,21.00,14.00,15.00,15.00,14.00,15.00
7,38.00,36.00,33.00,35.00,37.00,45.00,45.00,45.00,43.00,41.00,...,37.00,39.00,37.00,26.0,18.00,18.00,14.00,13.00,13.00,13.00
8,65.00,66.00,53.00,39.00,29.00,32.00,22.00,27.00,27.00,20.00,...,8.00,13.00,28.00,39.0,45.00,54.00,66.00,50.00,32.00,22.00
9,39.00,18.00,19.00,23.00,34.00,30.00,32.00,36.00,29.00,16.00,...,1.00,7.00,24.00,37.0,49.00,37.00,28.00,17.00,24.00,29.00


In [14]:
all_train_data2[:10]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-1.355677,0.779179,0.375919,0.569287,-0.541475,0.949327,0.627186,-0.850401,0.506851,0.275214,-0.098084,0.282219,-0.530081,0.881810,-1.246312,-1.303972,-0.842156,-1.141225
1,-1.355677,0.779179,0.066870,0.091480,-0.672929,-0.149654,-0.323041,-0.101878,0.278038,1.055407,-0.098084,-0.391365,-0.420006,0.881810,-0.797207,-0.839334,-0.466713,-0.764819
2,-1.355677,0.779179,0.005060,-0.099643,-0.716748,-0.311268,-0.468215,-0.262276,0.201767,0.875363,-0.098084,-0.466208,-0.585118,0.881810,-1.037426,-1.648226,-1.217599,-1.047123
3,-1.514655,0.779179,-0.056750,-0.195204,-0.190929,-0.521367,-0.481412,-0.476140,-0.293994,0.815348,-0.098084,0.057691,-0.640156,0.331872,-0.838984,-1.092773,-1.593042,-1.329428
4,-1.673634,0.779179,-0.118560,-0.290765,-0.147111,-0.537529,-0.494610,-0.422674,-0.675349,0.575288,-0.098084,-0.091994,-0.475043,0.331872,-0.483877,-0.680935,-0.560574,-1.047123
5,-1.673634,0.779179,-0.273085,-0.768572,-0.278565,-1.022373,-0.916933,-0.208810,-1.171110,0.395244,-0.098084,-0.017151,-0.750231,-0.218066,-0.525655,-0.448616,0.190312,0.176195
6,-1.673634,0.779179,-0.056750,-0.386327,-0.103292,-0.521367,-0.455017,-0.422674,-1.476194,0.215199,-0.098084,0.057691,-0.695193,0.331872,-0.577876,-0.554216,0.190312,0.270297
7,-1.673634,0.779179,0.252299,-0.099643,0.028162,-0.375914,-0.309844,-0.529605,-1.552465,-0.084875,-0.098084,-0.017151,-0.640156,0.331872,-0.546543,-0.680935,-0.278991,0.270297
8,-1.196698,0.779179,1.210353,1.142655,1.956165,0.787712,1.287066,-0.583071,-1.209246,-0.144890,-0.098084,-0.541051,1.286159,1.431748,-0.337657,0.776338,-1.593042,-1.141225
9,-0.878741,0.779179,1.086733,0.855971,2.525802,1.757400,2.210899,-0.155344,-0.179588,0.515273,-0.098084,-1.289478,6.734880,0.881810,-1.152313,-0.057898,-1.405321,-1.329428


In [15]:
all_train_data2.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03
mean,2.921351e-15,-4.132630e-13,-1.393800e-14,5.078230e-15,3.474738e-15,-2.338465e-15,-3.438183e-15,-4.431640e-16,2.485088e-16,1.855094e-16,4.130921e-15,-9.199431e-16,5.026265e-16,3.205650e-13,8.574353e-17,5.245418e-16,5.851993e-16,1.040902e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-5.536816e+00,-1.518692e+01,-1.571093e+00,-1.341940e+00,-1.417839e+00,-1.636509e+00,-1.933149e+00,-1.705857e+00,-1.628736e+00,-1.345187e+00,-9.808389e-02,-3.310232e+00,-2.401358e+00,-1.121682e+01,-1.631707e+00,-1.673570e+00,-2.156207e+00,-1.611732e+00
25%,-7.197621e-01,-8.174309e-01,-4.276098e-01,-6.730107e-01,-4.538382e-01,-6.829821e-01,-6.529812e-01,-7.434694e-01,-7.134846e-01,-7.450388e-01,-9.808389e-02,-6.907360e-01,-6.401558e-01,-7.680033e-01,-8.807613e-01,-8.710141e-01,-7.482952e-01,-7.648192e-01
50%,7.513122e-02,-1.912605e-02,-1.494652e-01,-2.907653e-01,-2.347470e-01,-2.466224e-01,-2.966459e-01,-1.553437e-01,-1.795877e-01,-2.049049e-01,-9.808389e-02,1.325340e-01,-2.548928e-01,-2.180656e-01,-3.898786e-01,-3.905364e-01,-1.851306e-01,-2.002106e-01
75%,7.110459e-01,7.791788e-01,1.904894e-01,3.781641e-01,7.198057e-02,4.644824e-01,3.632342e-01,5.397139e-01,5.449866e-01,4.702623e-01,-9.808389e-02,8.061186e-01,3.505206e-01,3.318721e-01,5.918869e-01,6.390586e-01,5.657555e-01,5.526008e-01
max,2.141854e+00,2.375788e+00,2.219482e+01,1.108104e+01,1.264781e+01,5.797768e+00,7.753892e+00,1.064478e+01,5.273788e+00,5.436493e+00,3.607990e+01,1.928760e+00,1.058751e+01,6.381187e+00,2.127201e+00,2.128011e+00,5.071072e+00,4.975368e+00


In [16]:
from collections import Counter

In [17]:
n_date = len(Counter(train_data.iloc[:, 0]).keys())

In [18]:
x=np.empty((12*479,18))
y=np.empty((12*479,1))

In [19]:
year_data3={}
for m in range(12):
    month_data3=np.empty((18,480))
    month_data3[:,:]=all_train_data1.T.iloc[:,m*480:(m+1)*480]
    year_data3[m]=month_data3

In [20]:
x=np.empty((12*471,18*9))
y=np.empty((12*471,1))

In [21]:
year_data3[0].shape

(18, 480)

In [22]:
year_data3[0][:,:5]

array([[ 14.  ,  14.  ,  14.  ,  13.  ,  12.  ],
       [  1.8 ,   1.8 ,   1.8 ,   1.8 ,   1.8 ],
       [  0.51,   0.41,   0.39,   0.37,   0.35],
       [  0.2 ,   0.15,   0.13,   0.12,   0.11],
       [  0.9 ,   0.6 ,   0.5 ,   1.7 ,   1.8 ],
       [ 16.  ,   9.2 ,   8.2 ,   6.9 ,   6.8 ],
       [ 17.  ,   9.8 ,   8.7 ,   8.6 ,   8.5 ],
       [ 16.  ,  30.  ,  27.  ,  23.  ,  24.  ],
       [ 56.  ,  50.  ,  48.  ,  35.  ,  25.  ],
       [ 26.  ,  39.  ,  36.  ,  35.  ,  31.  ],
       [  0.  ,   0.  ,   0.  ,   0.  ,   0.  ],
       [ 77.  ,  68.  ,  67.  ,  74.  ,  72.  ],
       [  1.8 ,   2.  ,   1.7 ,   1.6 ,   1.9 ],
       [  2.  ,   2.  ,   2.  ,   1.9 ,   1.9 ],
       [ 37.  ,  80.  ,  57.  ,  76.  , 110.  ],
       [ 35.  ,  79.  ,   2.4 ,  55.  ,  94.  ],
       [  1.4 ,   1.8 ,   1.  ,   0.6 ,   1.7 ],
       [  0.5 ,   0.9 ,   0.6 ,   0.3 ,   0.6 ]])

In [23]:
for m in range(12):
    for g in range(471):
        x[m*471+g:m*471+g+1,:]=year_data3[m][:,g:g+9].reshape(1,-1)
        y[m*471+g:m*471+g+1,:]=year_data3[m][9,9+g]

In [24]:
x[0]

array([1.40e+01, 1.40e+01, 1.40e+01, 1.30e+01, 1.20e+01, 1.20e+01,
       1.20e+01, 1.20e+01, 1.50e+01, 1.80e+00, 1.80e+00, 1.80e+00,
       1.80e+00, 1.80e+00, 1.80e+00, 1.80e+00, 1.80e+00, 1.80e+00,
       5.10e-01, 4.10e-01, 3.90e-01, 3.70e-01, 3.50e-01, 3.00e-01,
       3.70e-01, 4.70e-01, 7.80e-01, 2.00e-01, 1.50e-01, 1.30e-01,
       1.20e-01, 1.10e-01, 6.00e-02, 1.00e-01, 1.30e-01, 2.60e-01,
       9.00e-01, 6.00e-01, 5.00e-01, 1.70e+00, 1.80e+00, 1.50e+00,
       1.90e+00, 2.20e+00, 6.60e+00, 1.60e+01, 9.20e+00, 8.20e+00,
       6.90e+00, 6.80e+00, 3.80e+00, 6.90e+00, 7.80e+00, 1.50e+01,
       1.70e+01, 9.80e+00, 8.70e+00, 8.60e+00, 8.50e+00, 5.30e+00,
       8.80e+00, 9.90e+00, 2.20e+01, 1.60e+01, 3.00e+01, 2.70e+01,
       2.30e+01, 2.40e+01, 2.80e+01, 2.40e+01, 2.20e+01, 2.10e+01,
       5.60e+01, 5.00e+01, 4.80e+01, 3.50e+01, 2.50e+01, 1.20e+01,
       4.00e+00, 2.00e+00, 1.10e+01, 2.60e+01, 3.90e+01, 3.60e+01,
       3.50e+01, 3.10e+01, 2.80e+01, 2.50e+01, 2.00e+01, 1.90e

In [28]:
x[0,81:95]

array([26., 39., 36., 35., 31., 28., 25., 20., 19.,  0.,  0.,  0.,  0.,
        0.])

In [26]:
y[0]

array([30.])

In [45]:
data_idx=list(range(len(x)))
np.random.shuffle(data_idx)
x_train=x[data_idx[500:]]
y_train=y[data_idx[500:]]
x_test=x[data_idx[:500]]
y_test=y[data_idx[:500]]

In [37]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,cross_val_score

In [46]:
LR = linear_model.LinearRegression()
LR.fit(x_train, y_train)
y_pred = LR.predict(x_test)
print(mean_squared_error(y_pred, y_test))

29.8601378989349


In [33]:
x_train.shape

(5152, 162)

In [34]:
y_test.shape

(500, 1)

In [47]:
Lasso=linear_model.Lasso(alpha=0.00005)
Lasso.fit(x_train,y_train)
y_pred=Lasso.predict(x_test)
print(mean_squared_error(y_pred, y_test))

29.82480006749143


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [48]:
kf=KFold(n_splits=5,shuffle=True,random_state=7).split(x_train)
Lasso=linear_model.Lasso(alpha=0.00005)
score=-cross_val_score(Lasso,x_train,y_train,scoring='neg_mean_squared_error',cv=kf)
print('score',np.mean(score))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

score 36.216591681149694


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [49]:
kf=KFold(n_splits=9,shuffle=True,random_state=7).split(x_train)
Lasso=linear_model.Lasso(alpha=0.00005)
score=-cross_val_score(Lasso,x_train,y_train,scoring='neg_mean_squared_error',cv=kf)
print('score',np.mean(score))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

score 35.85285770403073


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


#### x1,y1 norm 

In [51]:
x1=np.empty((12*471,18*9))
y1=np.empty((12*471,1))

In [52]:
year_data4={}
for m in range(12):
    month_data4=np.empty((18,480))
    month_data4[:,:]=all_train_data2.T.iloc[:,m*480:(m+1)*480]
    year_data4[m]=month_data4

In [53]:
for m in range(12):
    for g in range(471):
        x1[m*471+g:m*471+g+1,:]=year_data4[m][:,g:g+9].reshape(1,-1)
        y1[m*471+g:m*471+g+1,:]=year_data4[m][9,9+g]

In [56]:
data_idx=list(range(len(x)))
np.random.shuffle(data_idx)
x_train1=x1[data_idx[500:]]
y_train1=y1[data_idx[500:]]
x_test1=x1[data_idx[:500]]
y_test1=y1[data_idx[:500]]

In [57]:
Lasso=linear_model.Lasso(alpha=0.00005)
Lasso.fit(x_train1,y_train1)
y_pred1=Lasso.predict(x_test1)
print(mean_squared_error(y_pred1, y_test1))

0.11451844837548714


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
